# Librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


# Constantes

In [ ]:
FILE_DATOS = './data/lagoon_hourly_filled.csv'

HOR_31 = 120
HOR_32 = 400
HOR_33 = 420

HORIZONTES_3 = [HOR_31, HOR_32, HOR_33]

PRIMER_CONJUNTO = 1110
SEGUNDO_CONJUNTO = 1500
TERCER_CONJUNTO = 2113

# Carga de datos

In [ ]:
parametro = 'temperatura'
output=parametro
inputs = [
          'fecha', 'temperatura',
          'ambiente', 'nivel'
          ]
usecols = inputs.copy()

datos = pd.read_csv(FILE_DATOS,
                    sep=',', 
                    usecols = usecols)

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%d/%m/%Y %H:%M')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('H')
datos = datos.sort_index()

datos.info()

datos.head()

# Modelos

In [ ]:
def separar_train_test(datos, conjunto, horizonte):
    data = datos[:conjunto].copy()
    steps = horizonte
    

    datos_train = data[:-steps]
    
    datos_test  = data[-steps:]

    print(f'Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})')
    print(f'Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})')

    fig, ax = plt.subplots(figsize=(20, 8))
    datos_train['nivel'].plot(ax=ax, label='entrenamiento')
    datos_test['nivel'].plot(ax=ax, label='test')
    ax.set_title(f'Conjunto de entrenamiento y test - Dataset {conjunto} [{horizonte} hora(s)]')
    ax.legend()
    plt.show()

    return datos_train, datos_test
    

In [ ]:
def obtener_metricas(test, predictions):
    r2 = np.NaN
    if len(test) > 1:
        r2 = r2_score(test, predictions)
    mse = mean_squared_error(test, predictions)
    mae = mean_absolute_error(test, predictions)

    return r2, mse, mae

In [ ]:
def pintar_resultados(df_ret, test, steps, modelo):
    fig, ax = plt.subplots(figsize=(15, 5))
    plt.title(f'{modelo} - [{steps} hora(s)]', fontsize=30);
    #df_train['temperatura'].plot(ax=ax, label='train')

    df_ret['Pred'].plot(ax=ax, label='Predicción')
    test.plot(ax=ax, label='Original')
    # datos['temperatura'].tail(25+steps).plot(ax=ax, label='Original')
    ax.legend()
    plt.show()

##  - Autoregression (AR)

In [ ]:
def AR_model(train,test,dataset, horizonte):
    # fit model
    model_AR = AutoReg(train['nivel'], lags=24)
    model_AR_fit = model_AR.fit()
    # make prediction
    predictions = model_AR_fit.predict(len(train), len(train) + len(test) - 1)
    res=pd.DataFrame({'Pred':predictions})
    pintar_resultados(res, test['nivel'], horizonte, 'AR')

    r2, mse, mae = obtener_metricas(test['nivel'].values, predictions)
    resultados = pd.DataFrame({'model':['AR'],
                               'data':[dataset],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados

## - SARIMA

In [ ]:
def SARIMA_model(train,test,dataset, horizonte):
    # fit model
    model_SARIMA = SARIMAX(train['nivel'], order=(3, 1, 1), seasonal_order=(0, 0, 2, 24))
    model__SARIMA_fit = model_SARIMA.fit(disp=False)
    # make prediction
    predictions = model__SARIMA_fit.predict(len(train), len(train) + len(test) -1 )
    
    res=pd.DataFrame({'Pred':predictions})
    pintar_resultados(res, test['nivel'], horizonte, 'SARIMA')
    
    r2, mse, mae = obtener_metricas(test['nivel'].values, predictions)
    resultados = pd.DataFrame({'model':['SARIMA'],
                               'data':[dataset],
                               'horizonte':[horizonte],
                               'r2':[r2], 
                               'mse':[mse], 
                               'mae':[mae]})
    return resultados

In [ ]:
def ejecutar_modelos(datos_train, datos_test, resultados, dataset, horizonte):
    res = AR_model(datos_train, datos_test, dataset, horizonte)
    resultados = pd.concat([resultados, res], axis=0)

    res = SARIMA_model(datos_train, datos_test, dataset, horizonte)
    resultados = pd.concat([resultados, res], axis=0)

    return resultados

# Resultados

In [ ]:
resultados = pd.DataFrame()


In [ ]:
# DATASET 3

for hor in HORIZONTES_3:
    datos_train, datos_test = separar_train_test(datos, TERCER_CONJUNTO, hor)
    resultados = ejecutar_modelos(datos_train, datos_test, resultados, 3, hor)

In [ ]:
resultados.reset_index(drop=True, inplace=True)
resultados

In [ ]:
datos_train["nivel"][-1]

In [ ]:
for hor in HORIZONTES_3:
    datos_train, datos_test = separar_train_test(datos, TERCER_CONJUNTO, hor)
    nivel_valor = datos_train["nivel"].iloc[-1]
    predictions = pd.Series([nivel_valor] * hor)

    r2, mse, mae = obtener_metricas(datos_test['nivel'].values, predictions)

    res = pd.DataFrame({'model':['HEREDANDO'],
                                'data':["3"],
                                'horizonte':[hor],
                                'r2':[r2], 
                                'mse':[mse], 
                                'mae':[mae]})
    resultados = pd.concat([resultados, res], axis=0)


In [ ]:
resultados.to_excel('resultados.xlsx', sheet_name='Experimentos_2',index=False)


In [ ]:
df = pd.DataFrame(resultados)
df = df.round(2)
latex_code = df.to_latex(index=False, float_format="%.2f")

In [ ]:
print(latex_code)